<a href="https://colab.research.google.com/github/pan7ae/scientific_work/blob/main/name.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Chest X-ray (Indiana University) datase from kaggle

In [ ]:
# Install kaggle
!pip install -q kaggle

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Change directory
import os
os.chdir("/content/drive/MyDrive/Nauchka")

In [ ]:
# Create a kaggle folder
!mkdir -p ~/.kaggle

# Copy kaggle.json to created folder
!cp kaggle.json ~/.kaggle/

In [ ]:
# Permission for the json to act
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download the required dataset
!kaggle datasets download -d raddar/chest-xrays-indiana-university

100% 13.2G/13.2G [03:47<00:00, 72.0MB/s]
100% 13.2G/13.2G [03:47<00:00, 62.1MB/s]


In [ ]:
# Unzip our dataset
from zipfile import ZipFile
from tqdm import tqdm


file_to_extract = "chest-xrays-indiana-university.zip"

# Open your .zip file
with ZipFile(file=file_to_extract) as zip_file:

    # Loop over each file and extract them
    for file in tqdm(iterable=zip_file.namelist(), total=len(zip_file.namelist())):
        zip_file.extract(member=file)

100%|██████████| 7472/7472 [08:35<00:00, 14.50it/s]


# Preprocessing the image and the text data

## Prepare photo data